In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import cvxpy as cp

# Generate training data (replace with your actual data)
# X_train: concatenation of state and control input
# Y_train: next state
X_train = np.random.rand(1000, 6)  # 4 state variables + 2 control inputs
Y_train = np.random.rand(1000, 4)  # 4 state variables

# Define and train the neural network
model = Sequential([
    Dense(64, activation='relu', input_shape=(6,)),
    Dense(64, activation='relu'),
    Dense(4)  # Output layer (next state)
])

model.compile(optimizer='adam', loss='mse')
model.fit(X_train, Y_train, epochs=1, batch_size=32)

# Save the model
model.save('dynamics_model.h5')

############### MPC with neural network dynamics##################
import numpy as np
import cvxpy as cp
from tensorflow.keras.models import load_model

# Load the trained neural network model
model = load_model('dynamics_model.h5')

# Define MPC parameters
dt = 0.1
N = 10
Q_y = 1.0
Q_psi = 1.0
R_delta = 0.1
R_a = 0.1

# Constraints
y_min, y_max = -1.0, 1.0
psi_min, psi_max = -np.pi / 4, np.pi / 4
delta_min, delta_max = -0.5, 0.5
a_min, a_max = -1.0, 1.0

# Initial state
x = np.array([0.0, 0.0, 0.0, 0.0])


def predict_next_state(current_state, control_input):
    input_data = np.concatenate((current_state, control_input)).reshape(1, -1)
    next_state = model.predict(input_data)
    return next_state.flatten()


def solve_mpc(x0):
    x = cp.Variable((4, N + 1))
    u = cp.Variable((2, N))

    cost = 0
    constraints = []

    for k in range(N):
        cost += cp.quad_form(x[:, k], np.diag([Q_y, 0, Q_psi, 0]))
        cost += cp.quad_form(u[:, k], np.diag([R_delta, R_a]))

        if k == 0:
            constraints += [x[:, k] == x0]
        else:
            predicted_state = predict_next_state(x[:, k - 1].value, u[:, k - 1].value)
            constraints += [x[:, k] == predicted_state]

        constraints += [y_min <= x[0, k], x[0, k] <= y_max]
        constraints += [psi_min <= x[2, k], x[2, k] <= psi_max]
        constraints += [delta_min <= u[0, k], u[0, k] <= delta_max]
        constraints += [a_min <= u[1, k], u[1, k] <= a_max]

    prob = cp.Problem(cp.Minimize(cost), constraints)
    prob.solve()

    return u[:, 0].value


# Simulation loop
for t in range(100):
    u_optimal = solve_mpc(x)
    x = predict_next_state(x, u_optimal)
    print(f"Time step {t}, State: {x}, Control: {u_optimal}")


In [3]:
import numpy as np
import cvxpy as cp
from tensorflow.keras.models import load_model
import tensorflow as tf

def custom_mse(y_true, y_pred):
    mask = tf.cast(tf.not_equal(y_true, -999), tf.float32)
    error = tf.square(y_pred - y_true) * mask
    return tf.reduce_sum(error) / tf.reduce_sum(mask)


In [4]:
from tensorflow.keras.models import load_model

model = load_model('dynamics_model.h5', custom_objects={'custom_mse': custom_mse})


In [12]:
# Define MPC parameters
dt = 0.1
N = 10 
Q_y = 1.0
Q_psi = 1.0
R_delta = 0.1
R_a = 0.1

# Constraints
y_min, y_max = -1.0, 1.0
psi_min, psi_max = -np.pi / 4, np.pi / 4
delta_min, delta_max = -0.5, 0.5
a_min, a_max = -1.0, 1.0

# Initial state
x = np.array([0.0, 0.0, 0.0, 0.0])

import numpy as np

def predict_next_state(current_state, control_input):
    current_state = np.array(current_state).flatten() 
    control_input = np.array(control_input).flatten()  
    input_data = np.concatenate((current_state, control_input)).reshape(1, -1)
    next_state = model.predict(input_data)
    return next_state.flatten()


def solve_mpc(x0):
    x = cp.Variable((4, N + 1))
    u = cp.Variable((2, N))

    cost = 0
    constraints = []

    for k in range(N):
        cost += cp.quad_form(x[:, k], np.diag([Q_y, 0, Q_psi, 0]))
        cost += cp.quad_form(u[:, k], np.diag([R_delta, R_a]))

        if k == 0:
            constraints += [x[:, k] == x0]
        else:
            
            current_state = x[:, k - 1].value
            control_input = u[:, k - 1].value
            if current_state is not None and control_input is not None:
                current_state = np.array(current_state, ndmin=1)  
                control_input = np.array(control_input, ndmin=1)  
                predicted_state = predict_next_state(current_state, control_input)
                constraints += [x[:, k] == predicted_state]

        constraints += [y_min <= x[0, k], x[0, k] <= y_max]
        constraints += [psi_min <= x[2, k], x[2, k] <= psi_max]
        constraints += [delta_min <= u[0, k], u[0, k] <= delta_max]
        constraints += [a_min <= u[1, k], u[1, k] <= a_max]

    prob = cp.Problem(cp.Minimize(cost), constraints)
    prob.solve()

    return u[:, 0].value

for t in range(100):
    u_optimal = solve_mpc(x)
    x = predict_next_state(x, u_optimal)
    print(f"Time step {t}, State: {x}, Control: {u_optimal}")


1/1 [==============================] - 0s 14ms/step
Time step 0, State: [0.02100139 0.02802369 0.02926531 0.04208713], Control: [0. 0.]
1/1 [==============================] - 0s 10ms/step
Time step 1, State: [0.0278432  0.02954761 0.03903926 0.06098496], Control: [-0. -0.]
1/1 [==============================] - 0s 10ms/step
Time step 2, State: [0.03010735 0.03062646 0.04334976 0.07021194], Control: [-0. -0.]
1/1 [==============================] - 0s 9ms/step
Time step 3, State: [0.03122116 0.03175335 0.04572025 0.07453418], Control: [-0. -0.]
1/1 [==============================] - 0s 10ms/step
Time step 4, State: [0.03136075 0.0327906  0.04721818 0.07736987], Control: [-0. -0.]
1/1 [==============================] - 0s 10ms/step
Time step 5, State: [0.03123144 0.03358233 0.04828958 0.07921878], Control: [-0. -0.]
1/1 [==============================] - 0s 10ms/step
Time step 6, State: [0.03107019 0.03414411 0.04903501 0.08042165], Control: [-0. -0.]
1/1 [==============================] 

1/1 [==============================] - 0s 10ms/step
Time step 59, State: [0.03064645 0.03529772 0.05053791 0.08270603], Control: [-0. -0.]
1/1 [==============================] - 0s 14ms/step
Time step 60, State: [0.03064646 0.03529772 0.05053791 0.08270603], Control: [-0. -0.]
1/1 [==============================] - 0s 15ms/step
Time step 61, State: [0.03064645 0.03529772 0.05053791 0.08270603], Control: [-0. -0.]
1/1 [==============================] - 0s 13ms/step
Time step 62, State: [0.03064646 0.03529772 0.05053791 0.08270603], Control: [-0. -0.]
1/1 [==============================] - 0s 10ms/step
Time step 63, State: [0.03064645 0.03529772 0.05053791 0.08270603], Control: [-0. -0.]
1/1 [==============================] - 0s 10ms/step
Time step 64, State: [0.03064646 0.03529772 0.05053791 0.08270603], Control: [-0. -0.]
1/1 [==============================] - 0s 9ms/step
Time step 65, State: [0.03064645 0.03529772 0.05053791 0.08270603], Control: [-0. -0.]
1/1 [=======================